Modelo CNN-LSTM com Otimização de Hiperparâmetros
funcionando com PyTorch e Optuna

In [19]:
import os
import logging
from sklearn.impute import SimpleImputer, KNNImputer
from datetime import datetime
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
import torch
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [20]:
import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import MAE, SMAPE
from lightning.pytorch.callbacks import EarlyStopping
import optuna
import matplotlib.pyplot as plt

# Carregando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'], low_memory=False)
df_original.rename(columns={'PM2.5': 'PM25'}, inplace=True)
df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'], format='%Y-%m-%d %H:%M:%S')
df_original.index = df_original['Data e Hora']
df_original.sort_index(inplace=True)

# Filtrando e pré-processando os dados
colunas_selecionadas = ['PM25']
df = df_original[colunas_selecionadas]
df = df.loc['2019-01-01':'2022-01-01']
df = df.apply(pd.to_numeric, errors='coerce')

# Lidando com valores ausentes
df_imputed = df.interpolate(method='linear').reset_index()

# Adicionando colunas temporais
df_imputed['time_idx'] = df_imputed.index
df_imputed['hour'] = df_imputed['Data e Hora'].dt.hour
df_imputed['dayofweek'] = df_imputed['Data e Hora'].dt.dayofweek
df_imputed['month'] = df_imputed['Data e Hora'].dt.month

# Criando um identificador de série
df_imputed['series_id'] = 0  # Apenas uma série temporal

# Definindo os comprimentos de entrada e previsão
max_encoder_length = 168  # 7 dias de dados históricos
max_prediction_length = 24  # Previsão de 1 dia

# Configurando o TimeSeriesDataSet
training_cutoff = df_imputed['time_idx'].max() - max_prediction_length
training = TimeSeriesDataSet(
    df_imputed[lambda x: x.time_idx <= training_cutoff],
    time_idx='time_idx',
    target='PM25',
    group_ids=['series_id'],
    static_categoricals=[],
    static_reals=[],
    time_varying_known_reals=['hour', 'dayofweek', 'month'],
    time_varying_unknown_reals=['PM25'],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
)

# Conjuntos de validação e teste
validation = TimeSeriesDataSet.from_dataset(training, df_imputed, predict=True, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

# Função para otimizar hiperparâmetros
def optimize_tft(trial):
    params = {
        "hidden_size": trial.suggest_int("hidden_size", 16, 128),
        "lstm_layers": trial.suggest_int("lstm_layers", 1, 5),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "hidden_continuous_size": trial.suggest_int("hidden_continuous_size", 8, 64),
        "attention_head_size": trial.suggest_int("attention_head_size", 1, 4),
    }

    model = TemporalFusionTransformer.from_dataset(
        training,
        **params
    )

    # Configurando o treinamento
    early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")
    trainer = pl.Trainer(
        max_epochs=100,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        gradient_clip_val=0.1,
        callbacks=[early_stop_callback],
    )
    trainer.fit(
        model,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    return trainer.callback_metrics["val_loss"].item()

# Otimização dos hiperparâmetros
study = optuna.create_study(direction="minimize")
study.optimize(optimize_tft, n_trials=3)

# Obtendo os melhores hiperparâmetros
best_params = study.best_params
print("Melhores hiperparâmetros:", best_params)

# Treinando o modelo final
final_model = TemporalFusionTransformer.from_dataset(training, **best_params)
early_stop_callback = EarlyStopping(monitor="val_loss", patience=50, mode="min")
trainer = pl.Trainer(
    max_epochs=500,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback],
)
trainer.fit(
    final_model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

# Salvando o modelo treinado
import os
os.makedirs('../best_models', exist_ok=True)
torch.save(final_model.state_dict(), '../best_models/best_tft_model.pth')

# Avaliação final
predictions = final_model.predict(val_dataloader)
actuals = torch.cat([y for x, y in iter(val_dataloader)])

# Cálculo de métricas
mae = MAE()(predictions, actuals)
smape = SMAPE()(predictions, actuals)
print(f"MAE: {mae:.4f}")
print(f"SMAPE: {smape:.4f}")

# Visualização dos resultados
preds = predictions.numpy()
acts = actuals.numpy()

plt.figure(figsize=(12, 6))
plt.plot(acts, label="Valores Reais")
plt.plot(preds, label="Previsões", alpha=0.7)
plt.legend()
plt.show()


[I 2024-11-24 15:21:52,980] A new study created in memory with name: no-name-9b561dca-647e-4522-8dd3-cedd3d33daf4
C:\dev\fast_api\venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
C:\dev\fast_api\venv\Lib\site-packages\lightning\pytorch\utilities\parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
C:\dev\fast_api\venv\Lib\site-packages\pytorch_forecasting\models\temporal_fusion_transformer\__init__.py:171: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default opt

C:\dev\fast_api\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
C:\dev\fast_api\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 204/204 [00:44<00:00,  4.54it/s, v_num=6, train_loss_step=2.780]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 204/204 [00:43<00:00,  4.68it/s, v_num=6, train_loss_step=2.660, val_loss=3.020, train_loss_epoch=3.620]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 204/204 [00:44<00:00,  4.63it/s, v_num=6, train_loss_step=2.400, val_loss=2.840, train_loss_epoch=2.840]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 204/204 [00:43<00:00,  4.66it/s, v_num=6, train_loss_step=2.470, val_loss=2.830, train_loss_epoch=2.710]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 204/204 [00:44<00:00,  4.61it/s, v_num=6, train_loss_step=2.470, val_loss=2.870, train_loss_epoch=2.670]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 204/204 [00:45<00:00,  4.50it/s, v_num=6, train_loss_step=2.500, val_loss=2.870, train_loss_epoch=2.630]
Validation: |          |

[I 2024-11-24 16:04:09,613] Trial 0 finished with value: 2.6091573238372803 and parameters: {'hidden_size': 67, 'lstm_layers': 1, 'dropout': 0.23758887934049186, 'learning_rate': 1.2054958057653017e-05, 'hidden_continuous_size': 28, 'attention_head_size': 2}. Best is trial 0 with value: 2.6091573238372803.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict                      | 184    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 0      | train
5  | e

Epoch 0: 100%|██████████| 204/204 [00:43<00:00,  4.74it/s, v_num=7, train_loss_step=4.170]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 204/204 [00:43<00:00,  4.74it/s, v_num=7, train_loss_step=3.140, val_loss=4.180, train_loss_epoch=4.890]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 204/204 [00:43<00:00,  4.73it/s, v_num=7, train_loss_step=2.860, val_loss=3.250, train_loss_epoch=3.570]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 204/204 [00:43<00:00,  4.73it/s, v_num=7, train_loss_step=3.110, val_loss=2.870, train_loss_epoch=3.060]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 204/204 [00:42<00:00,  4.75it/s, v_num=7, train_loss_step=2.640, val_loss=2.740, train_loss_epoch=2.850]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 204/204 [00:43<00:00,  4.74it/s, v_num=7, train_loss_step=2.900, val_loss=2.690, train_loss_epoch=2.760]
Validation: |          |

...


Melhores hiperparâmetros: {'hidden_size': 67, 'lstm_layers': 1, 'dropout': 0.23758887934049186, 'learning_rate': 1.2054958057653017e-05, 'hidden_continuous_size': 28, 'attention_head_size': 2}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict                      | 224    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 0      | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 24.4 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 18.2 K | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 18.4 K | train
8  | static_context_initial_hidden_lstm |

Epoch 0: 100%|██████████| 204/204 [00:43<00:00,  4.65it/s, v_num=9, train_loss_step=2.870]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 204/204 [00:44<00:00,  4.60it/s, v_num=9, train_loss_step=2.760, val_loss=3.200, train_loss_epoch=4.220]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 204/204 [00:43<00:00,  4.70it/s, v_num=9, train_loss_step=2.750, val_loss=2.990, train_loss_epoch=2.800]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 204/204 [00:43<00:00,  4.69it/s, v_num=9, train_loss_step=2.370, val_loss=2.990, train_loss_epoch=2.680]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 204/204 [00:43<00:00,  4.67it/s, v_num=9, train_loss_step=2.660, val_loss=2.980, train_loss_epoch=2.660]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 204/204 [00:43<00:00,  4.68it/s, v_num=9, train_loss_step=2.510, val_loss=2.960, train_loss_epoch=2.630]
Validation: |          |

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\dev\fast_api\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


TypeError: expected Tensor as element 0 in argument 0, but got tuple